# HOTEL BOOKING CANCELLATION TENDENCY 
## HYPER PARAMETER TUNING TO KNN, DECISION TREE AND RANDOM FOREST
### Notebook 2 of 2

[![hotel](https://www.journeyera.com/wp-content/uploads/2022/01/luxury-5-star-hotels-kathmandu-thamel-251085752.jpg)](https://www.journeyera.com/wp-content/uploads/2022/01/luxury-5-star-hotels-kathmandu-thamel-251085752.jpg)

Summary from `Notebook 1`:
1. We have perform project understanding and data wrangling to the dataset. 
2. Our **GOALS** is to reduce False-Negative cases (customer predicted to not cancel their bookings but they cancel in actual), means we need to build Model that produce highest Recall(+) score as possible.
3. Evaluation Matrix from Base Models : 
- ![evamatrix_base](Base_Model_Eva_Matrix.PNG)
4. Models benchmarking using cross-validation score (mean and standard deviation of Recall score) :
- ![crossval_base](Base_Model_Cross_Val.PNG)
5. Based on benchmark score, **K-Nearest Neighbors (KNN), Decision Tree and Random Forest** are 3 most potential Models, and those 3 will undergone tuning process in this Notebook.

In [2]:
## Import Common Package
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
## Import Dataset from Notebook 1
df=pd.read_csv('Clean_Data.csv', index_col=0)
df.head()

,previous_cancellations,booking_changes,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,is_canceled,country_Others,country_PRT,market_segment_Online TA,market_segment_Others,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reserved_room_type_A,reserved_room_type_Others
0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1,0
1,0,0,0,0,2,0,1,0,1,0,1,0,0,0,0,1,0,1,0
2,0,1,0,0,2,0,0,1,1,0,1,0,0,0,0,1,0,1,0
3,0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,0,1,0
4,0,2,0,0,2,0,0,1,1,0,1,0,0,0,0,1,0,1,0


We were given dataset by the project organizer (Purwadhika Digital Technology School), and the information in this dataset have been adjusted for the purpose of this project only. Dataset was given in .csv format and we have imported to this notebook as above. From this point, we will perform Data Understanding and Data Wrangling to understand more about the information in the dataset and preparing the information and format needed by Machine Learning model in the next stage of the project.

<hr>

## 5. HYPER PARAMETER TUNING USING GRIDSEARCH

We will maintain the same splitting parameters as used in Base Model.

### 5.1 Splitting Data

In [4]:
## Define features and target
x=df.drop(columns='is_canceled') ## all columns exclude target column
y=df['is_canceled'] ## target column

In [5]:
## Import Package for splitting data
from sklearn.model_selection import train_test_split

In [6]:
## Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.80, random_state=42, stratify=y)
## We split data into 80% training set : 20% testing set

In [7]:
## Check shape of features
print(x_train.shape, x_test.shape)

(66858, 18) (16715, 18)


### 5.2 Hyper Parameter Tuning for K-Nearest Neighbors (KNN)

In [8]:
## Import Package
from sklearn.neighbors import KNeighborsClassifier

In [9]:
## Build Model
KNN = KNeighborsClassifier()

In [10]:
## Fit Model
KNN.fit(x_train, y_train)

KNeighborsClassifier()


In KNN algorithm, there are several parameters to be set. It takes a point, finds the K-nearest points, and predicts a label for that point.
1. The first one is `number of K (examples)` closest to the query. Choosing the right K for our data is done by trying several Ks and picking the one that works best. Therefore, choosing a range of K-value is the best practice rather than manually try different numbers as K.
2. `Weights`
    - `Uniform` : algorithm will classify value based on total number of similar value in the data points. The biggest number of similar value in the proximity, prediction will be made to that value.
    - `Distance` : algorithm giving weight of distance of the value to similar value in the proximity. The closest the similar value to the actual value, algorithm will give more weight to the connection. Therefore, total number of similar value in the proximity did not always translate into predicted class, but rather how close they are to the predicted value.
3. `Power parameter`
    - p=1 : manhattan_distance (l1)
    - p=2 : euliddean_distance(l2)
    - arbitrary p : minkowski distance (l_p)

In [11]:
## Build Parameter for KNN Model
param_KNN={
    'n_neighbors':range(1,50,1), ## 1, 2, up to 49
    'weights':['uniform', 'distance'],
    'p':[1, 2]
}

GridSearch will automatically search for best score from every combinations available based on parameters inputted. For instance, total computation / iteration from parameters above are :

In [12]:
## Number of values in the parameters
print('Number of Neighbors:',len(range(1,50,1)))
print('Weights:', len(['uniform', 'distance']))
print('Power Parameters:', len([1,2]))

Number of Neighbors: 49
Weights: 2
Power Parameters: 2


In [13]:
## Total combination
print(' If we fold the features 3 times, number of combination available are:', 49 * 2 * 2 * 3)

 If we fold the features 3 times, number of combination available are: 588


In [14]:
## Import package
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [15]:
## Run and Fit Model
KNN_GS = GridSearchCV( KNN,
                        param_KNN,
                        cv = StratifiedKFold(n_splits=3),
                        n_jobs = 3,
                        verbose = 1, 
                        scoring = 'recall')
KNN_GS.fit(x_train,y_train)

Fitting 3 folds for each of 196 candidates, totalling 588 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=KNeighborsClassifier(), n_jobs=3,
             param_grid={'n_neighbors': range(1, 50), 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='recall', verbose=1)

In [16]:
## Recall Score based on best combination
KNN_GS.best_score_

0.848425616755471

In [17]:
## Best Parameter (combination) which produce best recall score above
KNN_GS.best_params_
## Number of neighbors is 1, using manhattan_distance and uniform weights

{'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}

In [18]:
## Save into variable
KNN_Tuned_GS = KNN_GS.best_estimator_

#### Evaluation Matrix for Tuned Model with K-Nearest Neighbors

In [19]:
## Import package
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score, f1_score

In [20]:
## Create Function to perform Evaluation Matrix scoring for Model
def Eva_Matrix(model,x_train,x_test,y_train,y_test,Nama):
    Model=model.fit(x_train,y_train)
    y_pred_train=Model.predict(x_train)
    acc_train=accuracy_score(y_train,y_pred_train)
    rec_train=recall_score(y_train,y_pred_train)
    prec_train=precision_score(y_train,y_pred_train)
    f1_train=f1_score(y_train,y_pred_train)

    y_pred_test=Model.predict(x_test)
    acc_test=accuracy_score(y_test,y_pred_test)
    rec_test=recall_score(y_test,y_pred_test)
    prec_test=precision_score(y_test,y_pred_test)
    f1_test=f1_score(y_test,y_pred_test)
    
    data = {
    Nama + ' Training':[acc_train,rec_train,prec_train,f1_train],
    Nama + ' Testing':[acc_test,rec_test,prec_test,f1_test]
}

    df=(pd.DataFrame(data,index=['Accuracy','Recall','Precision','F1']).T).round(4)
    cr_train=classification_report(y_train,y_pred_train)
    cm_train=confusion_matrix(y_train,y_pred_train,labels=[1,0])
    df_train=pd.DataFrame(data=cm_train,columns=['Pred 1','Pred 0'],index=['Akt 1','Akt 0'])

    cr_test=classification_report(y_test,y_pred_test)
    cm_test=confusion_matrix(y_test,y_pred_test,labels=[1,0])
    df_test=pd.DataFrame(data=cm_test,columns=['Pred 1','Pred 0'],index=['Akt 1','Akt 0'])

    return df,cr_train,df_train,cr_test,df_test

In [21]:
## Run function for K-Nearest Neighbors Model after hyper parameter tuning
df_KNN_Tuned_GS, cr_KNN_Tuned_GS_tr, cm_KNN_Tuned_GS_tr, cr_KNN_Tuned_GS_ts, cm_KNN_Tuned_GS_ts = Eva_Matrix( KNN_Tuned_GS, 
                                                                                                        x_train, x_test, 
                                                                                                        y_train, y_test,
                                                                                                        'KNN Tuned')

In [22]:
## Show score
df_KNN_Tuned_GS

,Accuracy,Recall,Precision,F1
KNN Tuned Training,0.737,0.8205,0.6055,0.6968
KNN Tuned Testing,0.736,0.8117,0.6057,0.6937


As we focused on Recall score for testing set, we could see that recall score for testing set of Model after tuning significantly higher, exceeding our ideal point of 0.80 for both training and testing set. We will compared to other Model after tuning whether this Model will outperform alternative Model or not.

In [23]:
## Show Classification Report
print(cr_KNN_Tuned_GS_tr, cr_KNN_Tuned_GS_ts)

              precision    recall  f1-score   support

           0       0.87      0.69      0.77     42236
           1       0.61      0.82      0.70     24622

    accuracy                           0.74     66858
   macro avg       0.74      0.75      0.73     66858
weighted avg       0.77      0.74      0.74     66858
               precision    recall  f1-score   support

           0       0.86      0.69      0.77     10559
           1       0.61      0.81      0.69      6156

    accuracy                           0.74     16715
   macro avg       0.73      0.75      0.73     16715
weighted avg       0.77      0.74      0.74     16715



As expected, Recall(+) reaching 0.82 for training set and 0.81 for testing set (improve by 0.1 point compared to before tuning).

In [24]:
## Show Confusion Matrix - Testing
cm_KNN_Tuned_GS_ts
## Result: number of False-Negative cases decrease significantly (one-third) compared to before tuning.

,Pred 1,Pred 0
Akt 1,4997,1159
Akt 0,3253,7306


### 5.3 Hyper Parameter Tuning for Decision Tree

In [25]:
## Import Package
from sklearn.tree import DecisionTreeClassifier

In [26]:
## Build Model
DT = DecisionTreeClassifier()

In [27]:
## Fit Model
DT.fit(x_train, y_train)

DecisionTreeClassifier()

There are several hyperparameters could be tuned in Decision Tree:
1. `max_depth` : The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than `min_samples_split` samples. The bigger, the deeper the tree.
2. `min_samples_split` : The minimum number of samples required to split an internal node (may be integer or float). The smaller, the deeper the tree.
3. `min_samples_leaf` : The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least `min_samples_leaf` training samples in each of the left and right branches. The smaller, the deeper the tree.
4. `max_features` : The number of features to consider when looking for the best split.

- [![DT](https://computersciencehub.io/wp-content/uploads/2021/09/Decision-Tree-Illustration.jpeg)](https://computersciencehub.io/wp-content/uploads/2021/09/Decision-Tree-Illustration.jpeg)

In [28]:
## Define Parameter
param_DT = {
    'max_depth' : np.arange(1, 20), ## 19 values
    'min_samples_split' : np.arange(2, 20), ## 18 values
    'min_samples_leaf' : np.arange(1, 20), ## 19 values
    'max_features' : np.arange(0.1, 1, 0.1) ## 9 values
}

In [29]:
## Number of values in the parameters
print('Max Depth:', len(np.arange(1,20)))
print('Min Samples Split:', len(np.arange(2,20)))
print('Min Samples Leaf:', len(np.arange(1,20)))
print('Max Features:', len(np.arange(0.1, 1, 0.1)))

Max Depth: 19
Min Samples Split: 18
Min Samples Leaf: 19
Max Features: 9


In [30]:
## Total combination
print(' If we fold the features 3 times, number of combination available are:', 19 * 18 * 19 * 9 * 3)

 If we fold the features 3 times, number of combination available are: 175446


If we use GridSearch for this computation, Machine will run 233,928 combinations and will put a lot of load to computer's processor. Meanwhile when using Randomized Search, we can determine how much iteration (samples) from the total combination, randomly, to be computed by computer.

** `RANDOMIZED SEARCH`** vs Grid Search
- Use Randomized Search if data are overwhelmingly huge
- Use Randomized Search if Algorithm has many hyper parameters
- GridSearch guaranteed best score from every possible combination, while Randomized Search didn't
- Computation for GridSearch is heavy because every combination must be tested, while Randomized Search randomly choose total of combination desired by user from every possible combination
- Fine Tuning on GridSearch perform by changing range of hyperparameter, while in Randomized Search we can change number of Iteration and change the range of hyperparameter at the same time.


In [31]:
## Import package
from sklearn.model_selection import RandomizedSearchCV

In [32]:
## Run Randomize Search
DT_RS = RandomizedSearchCV(DecisionTreeClassifier(),
                            param_DT,
                            cv = StratifiedKFold(n_splits=3), ## Number of Fold 
                            n_jobs = 3,
                            verbose = 1,
                            scoring = 'recall',
                            n_iter = 10000, ## Number of iteration
                            random_state = 42)
                        
## Fit Model
DT_RS.fit(x_train,y_train)

Fitting 3 folds for each of 10000 candidates, totalling 30000 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
                   estimator=DecisionTreeClassifier(), n_iter=10000, n_jobs=3,
                   param_distributions={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])},
                   random_state=42, scoring='recall', verbose=1)

In [33]:
## Recall Score based on best combination
DT_RS.best_score_

0.7195198309688721

In [34]:
## Best Parameter (combination) which produce best accuracy score above
DT_RS.best_params_
## Best parameters shown below:

{'min_samples_split': 16,
 'min_samples_leaf': 2,
 'max_features': 0.1,
 'max_depth': 15}

In [35]:
## Save into variable
DT_Tuned_RS = DT_RS.best_estimator_

#### Evaluation Matrix for Tuned Model with Decision Tree

In [36]:
## Run function for Decision Tree Model after hyper parameter tuning
df_DT_Tuned_RS, cr_DT_Tuned_RS_tr, cm_DT_Tuned_RS_tr, cr_DT_Tuned_RS_ts, cm_DT_Tuned_RS_ts = Eva_Matrix( DT_Tuned_RS, 
                                                                                                            x_train, x_test, 
                                                                                                            y_train, y_test,
                                                                                                            'Decision Tree Tuned')

In [37]:
## Show score
df_DT_Tuned_RS

,Accuracy,Recall,Precision,F1
Decision Tree Tuned Training,0.8071,0.7080,0.7535,0.7300
Decision Tree Tuned Testing,0.8039,0.6972,0.7522,0.7237


As we focused on Recall score for testing set, we could see that recall score for testing set of Model after tuning fall a bit below tuning and not exceeding our ideal point of 0.80 for both training and testing set. We had produce better score from KNN algorithm after hyperparameter tuning, we will not tune this Model further.

In [38]:
## Show Classification Report
print(cr_DT_Tuned_RS_tr, cr_DT_Tuned_RS_ts)

              precision    recall  f1-score   support

           0       0.84      0.86      0.85     42236
           1       0.75      0.71      0.73     24622

    accuracy                           0.81     66858
   macro avg       0.79      0.79      0.79     66858
weighted avg       0.81      0.81      0.81     66858
               precision    recall  f1-score   support

           0       0.83      0.87      0.85     10559
           1       0.75      0.70      0.72      6156

    accuracy                           0.80     16715
   macro avg       0.79      0.78      0.79     16715
weighted avg       0.80      0.80      0.80     16715



As expected, Recall(+) did not desirable since score for both training and testing set are below 0.80.

In [39]:
## Show Confusion Matrix - Testing
cm_DT_Tuned_RS_ts
## Result: number of False-Negative cases are still higher than KNN Model (after tuning)

,Pred 1,Pred 0
Akt 1,4292,1864
Akt 0,1414,9145


### 5.4 Hyper Parameter Tuning for Random Forest

In [40]:
## Import Package
from sklearn.ensemble import RandomForestClassifier

In [41]:
## Build Model
RF = RandomForestClassifier()

In [42]:
## Fit Model
RF.fit(x_train, y_train)

RandomForestClassifier()

Parameters in Random Forest to be tuned are similar to Decision Tree, but there is `n_estimators` as extra hyperparameters. `n_estimators` is number of tress in the forest. 

- [![RF](https://miro.medium.com/max/1400/1*hmtbIgxoflflJqMJ_UHwXw.jpeg)](https://miro.medium.com/max/1400/1*hmtbIgxoflflJqMJ_UHwXw.jpeg)

In [43]:
## Define Parameter
param_RF = {
    'n_estimators' : np.arange(0, 2001, 2), ## 1000 values
    'max_depth' : np.arange(1, 20), ## 19 values
    'min_samples_split' : np.arange(2, 20), ## 18 values
    'min_samples_leaf' : np.arange(1, 20), ## 19 values
    'max_features' : np.arange(0.1, 1, 0.1) ## 9 values
}

In [44]:
## Number of values in the parameters
print('Total Estimators:', len(np.arange(0, 2001, 2)))
print('Max Depth:', len(np.arange(1,20)))
print('Min Samples Split:', len(np.arange(2,20)))
print('Min Samples Leaf:', len(np.arange(1,20)))
print('Max Features:', len(np.arange(0.1, 1, 0.1)))

Total Estimators: 1001
Max Depth: 19
Min Samples Split: 18
Min Samples Leaf: 19
Max Features: 9


In [45]:
## Total combination
print(' If we fold the features 3 times, number of combination available are:', 1998 * 19 * 18 * 19 * 9 * 3)

 If we fold the features 3 times, number of combination available are: 350541108


If we use GridSearch for this computation, Machine will run 350,541,108 combinations and will put a lot of load to computer's processor. Meanwhile when using Randomized Search, we can determine how much iteration (samples) from the total combination, randomly, to be computed by computer.

In [46]:
## Run Randomize Search
RF_RS = RandomizedSearchCV(RandomForestClassifier(),
                            param_RF,
                            cv = StratifiedKFold(n_splits=3), ## Number of Fold 
                            n_jobs = 3,
                            verbose = 1,
                            scoring = 'recall',
                            n_iter = 300, ## Number of iteration
                            random_state = 42)
                        
## Fit Model
RF_RS.fit(x_train,y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
                   estimator=RandomForestClassifier(), n_iter=300, n_jobs=3,
                   param_distributions={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                                        'n_estimators': array([   0,    2,    4, ..., 1996, 1998, 2000])},
                   random_state=42, scoring='recall', verbose=1)

In [47]:
## Recall Score based on best combination
RF_RS.best_score_

0.7064413566193711

In [48]:
## Best Parameter (combination) which produce best recall score above
RF_RS.best_params_
## Number of neighbors is 1, using manhattan_distance and uniform

{'n_estimators': 468,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_features': 0.8,
 'max_depth': 17}

In [49]:
## Save into variable
RF_Tuned_RS = RF_RS.best_estimator_

#### Evaluation Matrix for Tuned Model with Random Forest

In [50]:
## Run function for Random Forest Model after hyper parameter tuning
df_RF_Tuned_RS, cr_RF_Tuned_RS_tr, cm_RF_Tuned_RS_tr, cr_RF_Tuned_RS_ts, cm_RF_Tuned_RS_ts = Eva_Matrix( RF_Tuned_RS, 
                                                                                                            x_train, x_test, 
                                                                                                            y_train, y_test,
                                                                                                            'Random Forest Tuned')

In [51]:
## Show score
df_RF_Tuned_RS

,Accuracy,Recall,Precision,F1
Random Forest Tuned Training,0.8096,0.7087,0.7585,0.7327
Random Forest Tuned Testing,0.8063,0.6982,0.7570,0.7264


As we focused on Recall score for testing set, we could see that recall score for testing set of Model after tuning significantly higher, exceeding our ideal point of 0.80 for both training and testing set. We will try for further improvement by using other algorithm and/or tuning parameter, however we expected great result in Confusion Matrix and Classification Report for confirmation.

In [52]:
## Show Classification Report
print(cr_RF_Tuned_RS_tr, cr_RF_Tuned_RS_ts)

              precision    recall  f1-score   support

           0       0.84      0.87      0.85     42236
           1       0.76      0.71      0.73     24622

    accuracy                           0.81     66858
   macro avg       0.80      0.79      0.79     66858
weighted avg       0.81      0.81      0.81     66858
               precision    recall  f1-score   support

           0       0.83      0.87      0.85     10559
           1       0.76      0.70      0.73      6156

    accuracy                           0.81     16715
   macro avg       0.79      0.78      0.79     16715
weighted avg       0.80      0.81      0.80     16715



As expected, Recall(+) did not desirable since score for both training and testing set are below 0.80.

In [56]:
## Show Confusion Matrix - Testing
cm_RF_Tuned_RS_ts
## Result: number of False-Negative cases are still higher than KNN Model (after tuning)

,Pred 1,Pred 0
Akt 1,4298,1858
Akt 0,1380,9179


<hr>

## 6. Summary After Hyperparameter Tuning

In [53]:
## Compare evaluation matrix of every Tuned Model
pd.concat([df_KNN_Tuned_GS, df_DT_Tuned_RS, df_RF_Tuned_RS])

,Accuracy,Recall,Precision,F1
KNN Tuned Training,0.7370,0.8205,0.6055,0.6968
KNN Tuned Testing,0.7360,0.8117,0.6057,0.6937
Decision Tree Tuned Training,0.8071,0.7080,0.7535,0.7300
Decision Tree Tuned Testing,0.8039,0.6972,0.7522,0.7237
Random Forest Tuned Training,0.8096,0.7087,0.7585,0.7327
Random Forest Tuned Testing,0.8063,0.6982,0.7570,0.7264


KNN Model with hyper parameters set above is the only algorithm that could produce both training and testing recall score more than 0.80. Since the result has reached our expectation of 0.80, therefore we will use this algorithm for prediction.

In [54]:
## Compare Confusion Matrix for Testing Set of every Tuned Model
print('Confusion Matrix - Testing using KNN Tuned')
print(cm_KNN_Tuned_GS_ts)
print('='*30)

print('Confusion Matrix - Testing using Decision Tree Tuned')
print(cm_DT_Tuned_RS_ts)
print('='*30)

print('Confusion Matrix - Testing using Random Forest Tuned')
print(cm_RF_Tuned_RS_ts)
print('='*30)

Confusion Matrix - Testing using KNN Tuned
       Pred 1  Pred 0
Akt 1    4997    1159
Akt 0    3253    7306
Confusion Matrix - Testing using Decision Tree Tuned
       Pred 1  Pred 0
Akt 1    4292    1864
Akt 0    1414    9145
Confusion Matrix - Testing using Random Forest Tuned
       Pred 1  Pred 0
Akt 1    4298    1858
Akt 0    1380    9179


As expected, KNN produced the least False-Negative cases compared to other algorithms. At the start of the project, we have calculate potential benefits that could be produce from the implementation of Machine Learning, since False-Negative cases did not reduced to 0 (zero), we will calculate potential benefits from the False-Negative rates.

\begin{equation}
False (-) Rates = \frac{False-Negative} {(False-Negative) + (True-Positive)} = 1 - Recall(-)
\end{equation}

In [63]:
## Calculate False-Negative Rates
FalseNegativeRates = (1159)/(1159+4997)
## Expected Recoverable Income after Machine Learning Implementation
print(f'Recoverable Income per year:',14843.4 * (1-FalseNegativeRates) * 365)

Recoverable Income per year: 4397815.37962963


- Cost of implementation of Machine Learning per year (stated at the start of project) : USD 629,790 per year

In [64]:
print(f'Expected benefit per year from implementation of Machine Learning: USD', 4397815.37962963 - 629790)

Expected benefit per year from implementation of Machine Learning: USD 3768025.3796296297


<hr>

## 7. Recommendation and Limitation

### 7.1 Recommendation

- Implement Machine Learning with our Model (expected income to more than USD 3.7 million in a year)
- Review Hotel Cancellations Policy, adjust for strict approach so that number of cancellations will be reduced
- Directly contact waiting list customers in case of cancellations
- Make sure number of employees suffice and trained well for an always full-packed hotel


### 7.2 Limitation & Further Improvement

- High-end luxurious hotel (5-stars expected)
- High on demand (waiting list available)
- Provide Hotel Occupancy Rate, if available
- Provide Rate per Rooms and total rooms, if available
- Provide current Hotel Cancellation Policy, if available

<hr>

## 8. Saving Model

In [65]:
## Import package
import pickle

In [66]:
## Export Model with Write Binary (wb)
pickle.dump(KNN, open('Hotel_ModelKNN_v.1.0.pkl', 'wb') )

In [67]:
df.head()

,previous_cancellations,booking_changes,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,is_canceled,country_Others,country_PRT,market_segment_Online TA,market_segment_Others,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reserved_room_type_A,reserved_room_type_Others
0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1,0
1,0,0,0,0,2,0,1,0,1,0,1,0,0,0,0,1,0,1,0
2,0,1,0,0,2,0,0,1,1,0,1,0,0,0,0,1,0,1,0
3,0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,0,1,0
4,0,2,0,0,2,0,0,1,1,0,1,0,0,0,0,1,0,1,0


<hr>

## Thank You